In [2]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
from PIL import ImageFont, ImageDraw, Image

# Test TensorFlow import
print("Testing TensorFlow import...")
import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor
from tensorflow.keras.models import load_model

print(f"✅ TensorFlow imported successfully!")
print(f"✅ TensorFlow version: {tf.__version__}")

# Test YOLO utilities
print("Testing YOLO utilities...")
from yad2k.models.keras_yolo import yolo_head
from yad2k.utils.utils import draw_boxes, get_colors_for_classes, scale_boxes, read_classes, read_anchors, preprocess_image

print("✅ YOLO utilities imported successfully!")
print("✅ Your environment is now fixed and ready to use!")

%matplotlib inline

Testing TensorFlow import...
✅ TensorFlow imported successfully!
✅ TensorFlow version: 2.19.0
Testing YOLO utilities...
✅ YOLO utilities imported successfully!
✅ Your environment is now fixed and ready to use!


In [3]:
def yolo_filter_boxes(boxes,box_confidence,box_class_probs,threshold=0.6):
    """Filters YOLO boxes by thresholding on object and class confidence.
    
    Arguments:
        boxes -- tensor of shape (19, 19, 5, 4)
        box_confidence -- tensor of shape (19, 19, 5, 1)
        box_class_probs -- tensor of shape (19, 19, 5, 80)
        threshold -- real value, if [ highest class probability score < threshold],
                     then get rid of the corresponding box

    Returns:
        scores -- tensor of shape (None,), containing the class probability score for selected boxes
        boxes -- tensor of shape (None, 4), containing (b_x, b_y, b_h, b_w) coordinates of selected boxes
        classes -- tensor of shape (None,), containing the index of the class detected by the selected boxes

    Note: "None" is here because you don't know the exact number of selected boxes, as it depends on the threshold. 
    For example, the actual output size of scores would be (10,) if there are 10 boxes.
    """
    x = 10
    y = tf.constant(100)

    # Step 1: Compute box scores
    box_scores = box_class_probs * box_confidence
    
    # Step 2: Get best class for each box
    # Pick the class index (0 to 79) with the highest score for each box.
    box_classes = tf.math.argmax(box_scores, axis=-1)
    # Get the actual score value (not index) of the best class for each box.
    box_class_scores = tf.math.reduce_max(box_scores, axis=-1)

    # Step 3: Create a mask for filtering
    filtering_mask = (box_class_scores >= threshold)

    # Step 4: Apply the mask (keep only good boxes)
    # Keep only scores of good boxes
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    
    # Keep only coordinates of good boxes
    boxes = tf.boolean_mask(boxes, filtering_mask)
    
    # Keep only class indices of good boxes
    classes = tf.boolean_mask(box_classes, filtering_mask)

    # Return the final filtered results
    return scores, boxes, classes

In [4]:
tf.random.set_seed(10)
box_confidence = tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1)
boxes = tf.random.normal([19, 19, 5, 4], mean=1, stddev=4, seed = 1)
box_class_probs = tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1)
scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = 0.5)
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.shape))
print("boxes.shape = " + str(boxes.shape))
print("classes.shape = " + str(classes.shape))

assert type(scores) == EagerTensor, "Use tensorflow functions"
assert type(boxes) == EagerTensor, "Use tensorflow functions"
assert type(classes) == EagerTensor, "Use tensorflow functions"

assert scores.shape == (1789,), "Wrong shape in scores"
assert boxes.shape == (1789, 4), "Wrong shape in boxes"
assert classes.shape == (1789,), "Wrong shape in classes"

assert np.isclose(scores[2].numpy(), 9.270486), "Values are wrong on scores"
assert np.allclose(boxes[2].numpy(), [4.6399336, 3.2303846, 4.431282, -2.202031]), "Values are wrong on boxes"
assert classes[2].numpy() == 8, "Values are wrong on classes"

print("\033[92m All tests passed!")

scores[2] = 9.270486
boxes[2] = [ 4.6399336  3.2303846  4.431282  -2.202031 ]
classes[2] = 8
scores.shape = (1789,)
boxes.shape = (1789, 4)
classes.shape = (1789,)
 All tests passed!


In [5]:
def iou(box1, box2):
 """
 Implement the intersection over union (IoU) between box1 and box2
    
    Arguments:
    box1 -- first box, list object with coordinates (box1_x1, box1_y1, box1_x2, box_1_y2)
    box2 -- second box, list object with coordinates (box2_x1, box2_y1, box2_x2, box2_y2)
 """
# Unpack the box coordinates
# Extract each coordinate from the lists to make code cleaner.

 (box1_x1, box1_y1, box1_x2, box1_y2) = box1
 (box2_x1, box2_y1, box2_x2, box2_y2) = box2

# Step 1: Find the Area intersection box

 xi1 = max(box1_x1, box2_x1)  # left of the overlapping area
 yi1 = max(box1_y1, box2_y1)  # top of the overlapping area
 xi2 = min(box1_x2, box2_x2)  # right of the overlapping area
 yi2 = min(box1_y2, box2_y2)  # bottom of the overlapping area

#These lines calculate the area where the two boxes overlap.
#max(0, ...) prevents negative area (if boxes don’t overlap at all).

 inter_width = max(0, yi2 - yi1)   # height of overlap
 inter_height = max(0, xi2 - xi1)  # width of overlap
 inter_area = inter_width * inter_height

# Step 2: Find the union area
 box1_area = (box1_x2 - box1_x1) * (box1_y2 - box1_y1)
 box2_area = (box2_x2 - box2_x1) * (box2_y2 - box2_y1)
 union_area = box1_area + box2_area - inter_area

# Step 3: Calculate the IoU
 iou = inter_area / union_area

 return iou

In [6]:
## Test case 1: boxes intersect
box1 = (2, 1, 4, 3)
box2 = (1, 2, 3, 4)

print("iou for intersecting boxes = " + str(iou(box1, box2)))
assert iou(box1, box2) < 1, "The intersection area must be always smaller or equal than the union area."
assert np.isclose(iou(box1, box2), 0.14285714), "Wrong value. Check your implementation. Problem with intersecting boxes"

## Test case 2: boxes do not intersect
box1 = (1,2,3,4)
box2 = (5,6,7,8)
print("iou for non-intersecting boxes = " + str(iou(box1,box2)))
assert iou(box1, box2) == 0, "Intersection must be 0"

## Test case 3: boxes intersect at vertices only
box1 = (1,1,2,2)
box2 = (2,2,3,3)
print("iou for boxes that only touch at vertices = " + str(iou(box1,box2)))
assert iou(box1, box2) == 0, "Intersection at vertices must be 0"

## Test case 4: boxes intersect at edge only
box1 = (1,1,3,3)
box2 = (2,3,3,4)
print("iou for boxes that only touch at edges = " + str(iou(box1,box2)))
assert iou(box1, box2) == 0, "Intersection at edges must be 0"

print("\033[92m All tests passed!")

iou for intersecting boxes = 0.14285714285714285
iou for non-intersecting boxes = 0.0
iou for boxes that only touch at vertices = 0.0
iou for boxes that only touch at edges = 0.0
 All tests passed!


In [7]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
 """
    Applies Non-max suppression (NMS) to set of boxes
    
    Arguments:
    scores -- tensor of shape (None,), output of yolo_filter_boxes()
    boxes -- tensor of shape (None, 4), output of yolo_filter_boxes() that have been scaled to the image size (see later)
    classes -- tensor of shape (None,), output of yolo_filter_boxes()
    max_boxes -- integer, maximum number of predicted boxes you'd like
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (, None), predicted score for each box
    boxes -- tensor of shape (4, None), predicted box coordinates
    classes -- tensor of shape (, None), predicted class for each box
    
    Note: The "None" dimension of the output tensors has obviously to be less than max_boxes. Note also that this
    function will transpose the shapes of scores, boxes, classes. This is made for convenience.
 """
 # Step 1: Prepare Tensor for NMS
 # TensorFlow needs the max box number as a tf.Variable, not plain int
 max_boxes_tensor = tf.Variable(max_boxes, dtype='int32')

 # Step 2: Apply Non-Max Suppression
 # This function looks at boxes with high scores
 # Keeps the best one in overlapping areas
 # Removes the rest if they overlap too much (IoU ≥ iou_threshold)
 # Returns nms_indices: indexes of boxes you should keep
 nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_threshold)

# Step 3: Select Only Best Boxes
# tf.gather() picks values at the chosen indices
 scores = tf.gather(scores, nms_indices)
 boxes = tf.gather(boxes, nms_indices)
 classes = tf.gather(classes, nms_indices)

# These are the final boxes, cleaned from duplicates and sorted by quality.
 return scores, boxes, classes

In [8]:
tf.random.set_seed(10)
scores = tf.random.normal([54,], mean=1, stddev=4, seed = 1)
boxes = tf.random.normal([54, 4], mean=1, stddev=4, seed = 1)
classes = tf.random.normal([54,], mean=1, stddev=4, seed = 1)
scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)

assert type(scores) == EagerTensor, "Use tensoflow functions"
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.numpy().shape))
print("boxes.shape = " + str(boxes.numpy().shape))
print("classes.shape = " + str(classes.numpy().shape))

assert type(scores) == EagerTensor, "Use tensoflow functions"
assert type(boxes) == EagerTensor, "Use tensoflow functions"
assert type(classes) == EagerTensor, "Use tensoflow functions"

assert scores.shape == (10,), "Wrong shape"
assert boxes.shape == (10, 4), "Wrong shape"
assert classes.shape == (10,), "Wrong shape"

assert np.isclose(scores[2].numpy(), 8.147684), "Wrong value on scores"
assert np.allclose(boxes[2].numpy(), [ 6.0797963, 3.743308, 1.3914018, -0.34089637]), "Wrong value on boxes"
assert np.isclose(classes[2].numpy(), 1.7079165), "Wrong value on classes"

print("\033[92m All tests passed!")

scores[2] = 8.147684
boxes[2] = [ 6.0797963   3.743308    1.3914018  -0.34089637]
classes[2] = 1.7079165
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)
 All tests passed!


In [9]:
def yolo_boxes_to_corners(box_xy, box_wh):
 """
 YOLO gives boxes using:

box_xy: center of the box → (x, y)

box_wh: width and height of the box → (w, h)

But for drawing or calculating overlap, we need corners:

Top-left: (x_min, y_min)

Bottom-right: (x_max, y_max)

This function converts center format to corner format
 """
 # Step 1: Calculate box corners
 # You go half the width/height left and right from the center to get edges.
 box_mins = box_xy - (box_wh / 2.)
 box_maxes = box_xy + (box_wh / 2.)
 
 # Step 2: Reorder values to [y_min, x_min, y_max, x_max]
 return tf.keras.backend.concatenate([
    box_mins[..., 1:2],  # y_min
    box_mins[..., 0:1],  # x_min
    box_maxes[..., 1:2],  # y_max
    box_maxes[..., 0:1]  # x_max
]) 

In [10]:
def yolo_eval(yolo_outputs, image_shape = (720, 1280), max_boxes=10, score_threshold=.6, iou_threshold=.5):
  """
    Converts the output of YOLO encoding (a lot of boxes) to your predicted boxes along with their scores, box coordinates and classes.
    
    Arguments:
    yolo_outputs -- output of the encoding model (for image_shape of (608, 608, 3)), contains 4 tensors:
                    box_xy: tensor of shape (None, 19, 19, 5, 2)
                    box_wh: tensor of shape (None, 19, 19, 5, 2)
                    box_confidence: tensor of shape (None, 19, 19, 5, 1)
                    box_class_probs: tensor of shape (None, 19, 19, 5, 80)
    image_shape -- tensor of shape (2,) containing the input shape, in this notebook we use (608., 608.) (has to be float32 dtype)
    max_boxes -- integer, maximum number of predicted boxes you'd like
    score_threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (None, ), predicted score for each box
    boxes -- tensor of shape (None, 4), predicted box coordinates
    classes -- tensor of shape (None,), predicted class for each box
  """

  # These are 4 tensors YOLO gives:
  # box_xy: center of each box (x, y)
  # box_wh: width and height of each box
  # box_confidence: how sure YOLO is that there’s an object
  # box_class_probs: how sure YOLO is about the object type (80 classes)
  box_xy, box_wh, box_confidence, box_class_probs = yolo_outputs

# Step 2: Convert to corner format
# YOLO gives center points + width/height.
# We convert that to corners: [y_min, x_min, y_max, x_max] for each box (needed for drawing and overlap calculation).
  boxes = yolo_boxes_to_corners(box_xy, box_wh)

# Step 3: Filter boxes using score threshold
# Removes boxes that don’t have high enough confidence.
# Keeps boxes where the highest class score is ≥ score_threshold (0.6)
  scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, score_threshold)

# Step 4: Scale boxes to original image size
# YOLO works on 608×608 images.
# But your real image might be 720×1280.
# This step resizes boxes to match the actual image size.
  boxes = scale_boxes(boxes, image_shape)

# Step 5: Remove overlapping boxes using Non-Max Suppression
# Keeps only the best boxes in overlapping areas.
# max_boxes: max number of final boxes to return
# iou_threshold: overlap limit (if too much overlap → discard)
  scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)

  return scores, boxes, classes

In [11]:
tf.random.set_seed(10)
yolo_outputs = (tf.random.normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                tf.random.normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))
scores, boxes, classes = yolo_eval(yolo_outputs)
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.numpy().shape))
print("boxes.shape = " + str(boxes.numpy().shape))
print("classes.shape = " + str(classes.numpy().shape))

assert type(scores) == EagerTensor, "Use tensoflow functions"
assert type(boxes) == EagerTensor, "Use tensoflow functions"
assert type(classes) == EagerTensor, "Use tensoflow functions"

assert scores.shape == (10,), "Wrong shape"
assert boxes.shape == (10, 4), "Wrong shape"
assert classes.shape == (10,), "Wrong shape"
    
assert np.isclose(scores[2].numpy(), 171.60194), "Wrong value on scores"
assert np.allclose(boxes[2].numpy(), [-1240.3483, -3212.5881, -645.78, 2024.3052]), "Wrong value on boxes"
assert np.isclose(classes[2].numpy(), 16), "Wrong value on classes"
    
print("\033[92m All tests passed!")

scores[2] = 171.60194
boxes[2] = [-1240.3483 -3212.5881  -645.78    2024.3052]
classes[2] = 16
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)
 All tests passed!


In [15]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
model_image_size = (608, 608) # Same as yolo_model input layer size

In [18]:
# Load YOLO model - comprehensive approach for different file formats
print("=== YOLO Model Loading ===")

# First, check if we have a working H5 file
import os
h5_file = "model_data/yolo.h5"
savedmodel_dir = "model_data"

if os.path.exists(h5_file):
    try:
        print("Found yolo.h5 file, attempting to load...")
        yolo_model = load_model(h5_file, compile=False)
        print("✅ Successfully loaded yolo.h5 model!")
        model_loaded = True
    except Exception as e:
        print(f"❌ H5 file is corrupted: {e}")
        model_loaded = False
else:
    print("❌ No yolo.h5 file found")
    model_loaded = False

# If H5 didn't work, try SavedModel approaches
if not model_loaded:
    try:
        print("Attempting SavedModel with TFSMLayer...")
        yolo_model = tf.keras.layers.TFSMLayer(savedmodel_dir, call_endpoint='serving_default')
        print("✅ Successfully loaded SavedModel with TFSMLayer!")
        model_loaded = True
    except Exception as e:
        print(f"❌ TFSMLayer failed: {e}")
        
        try:
            print("Attempting direct SavedModel load...")
            yolo_model = tf.saved_model.load(savedmodel_dir)
            print("✅ Successfully loaded SavedModel directly!")
            model_loaded = True
        except Exception as e2:
            print(f"❌ Direct SavedModel load failed: {e2}")
            model_loaded = False

# If nothing worked, create a placeholder and explain the issue
if not model_loaded:
    print("\n🔧 Creating placeholder model...")
    print("📋 Model files appear to be incomplete due to Git LFS issues")
    
    # Create a simple placeholder
    input_layer = tf.keras.Input(shape=(608, 608, 3))
    x = tf.keras.layers.GlobalAveragePooling2D()(input_layer)
    output_layer = tf.keras.layers.Dense(1, activation='linear')(x)
    yolo_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    
    print("✅ Placeholder model created")
    print("\n📖 How to fix the model files:")
    print("1. Install Git LFS: git lfs install")
    print("2. Download actual files: git lfs pull")
    print("3. Or download YOLO weights manually from the original source")
    print("\n💡 Your YOLO algorithm functions still work perfectly!")
    print("   The model file is only needed for inference on real images.")

print("\n✅ Model loading process completed!")

=== YOLO Model Loading ===
❌ No yolo.h5 file found
Attempting SavedModel with TFSMLayer...
❌ TFSMLayer failed: NewRandomAccessFile failed to Create/Open: model_data\variables\variables.data-00000-of-00001 : The system cannot find the file specified.
; No such file or directory
Attempting direct SavedModel load...
❌ TFSMLayer failed: NewRandomAccessFile failed to Create/Open: model_data\variables\variables.data-00000-of-00001 : The system cannot find the file specified.
; No such file or directory
Attempting direct SavedModel load...
❌ Direct SavedModel load failed: NewRandomAccessFile failed to Create/Open: model_data\variables\variables.data-00000-of-00001 : The system cannot find the file specified.
; No such file or directory

🔧 Creating placeholder model...
📋 Model files appear to be incomplete due to Git LFS issues
✅ Placeholder model created

📖 How to fix the model files:
1. Install Git LFS: git lfs install
2. Download actual files: git lfs pull
3. Or download YOLO weights manual

In [19]:
# 🎉 Your YOLO Implementation Status

print("=== YOLO Lab Implementation Summary ===")
print()
print("✅ WORKING COMPONENTS:")
print("   • TensorFlow and Keras properly installed")
print("   • All YOLO utility functions imported")
print("   • yolo_filter_boxes() - Filters boxes by confidence")
print("   • iou() - Calculates Intersection over Union")
print("   • yolo_non_max_suppression() - Removes duplicate detections")
print("   • yolo_boxes_to_corners() - Converts box formats")
print("   • yolo_eval() - Complete YOLO evaluation pipeline")
print()
print("🔧 PLACEHOLDER CREATED:")
print("   • Model structure exists (for testing/learning)")
print("   • All your algorithm functions work correctly")
print()
print("📋 WHAT YOU CAN DO NOW:")
print("   1. Test all your YOLO functions with synthetic data")
print("   2. Run the test cells to verify implementations")
print("   3. Study the YOLO algorithm concepts")
print("   4. Practice with the evaluation pipeline")
print()
print("🚀 TO USE WITH REAL IMAGES:")
print("   • Fix the Git LFS issue to get actual model weights")
print("   • Or download pre-trained YOLO weights manually")
print("   • Your algorithm implementations are ready!")
print()
print("✨ Great job implementing the YOLO algorithm! ✨")

# Test if the basic structure works
print("\n=== Quick Test ===")
try:
    # Test with dummy data
    dummy_input = tf.random.normal([1, 608, 608, 3])
    output = yolo_model(dummy_input)
    print(f"✅ Model accepts input shape: {dummy_input.shape}")
    print(f"✅ Model produces output shape: {output.shape}")
    print("✅ Basic model structure is working!")
except Exception as e:
    print(f"❌ Model test failed: {e}")
    print("This is expected with the placeholder model")

=== YOLO Lab Implementation Summary ===

✅ WORKING COMPONENTS:
   • TensorFlow and Keras properly installed
   • All YOLO utility functions imported
   • yolo_filter_boxes() - Filters boxes by confidence
   • iou() - Calculates Intersection over Union
   • yolo_non_max_suppression() - Removes duplicate detections
   • yolo_boxes_to_corners() - Converts box formats
   • yolo_eval() - Complete YOLO evaluation pipeline

🔧 PLACEHOLDER CREATED:
   • Model structure exists (for testing/learning)
   • All your algorithm functions work correctly

📋 WHAT YOU CAN DO NOW:
   1. Test all your YOLO functions with synthetic data
   2. Run the test cells to verify implementations
   3. Study the YOLO algorithm concepts
   4. Practice with the evaluation pipeline

🚀 TO USE WITH REAL IMAGES:
   • Fix the Git LFS issue to get actual model weights
   • Or download pre-trained YOLO weights manually
   • Your algorithm implementations are ready!

✨ Great job implementing the YOLO algorithm! ✨

=== Quick Te

In [21]:
# 📊 YOLO Model Summary

print("=== YOLO Model Architecture Summary ===")
print()

# Check what type of model we have
print("🔍 Model Type Analysis:")
print(f"   • Model class: {type(yolo_model)}")
print(f"   • Model name: {getattr(yolo_model, 'name', 'Not available')}")
print()

# Try to get model summary
try:
    print("📋 Model Summary:")
    print("-" * 50)
    yolo_model.summary()
    print("-" * 50)
    
except Exception as e:
    print(f"❌ Could not display summary: {e}")
    print("This might be due to the placeholder model structure")

print()

# Display input/output shapes
try:
    print("📐 Input/Output Information:")
    if hasattr(yolo_model, 'input_shape'):
        print(f"   • Input shape: {yolo_model.input_shape}")
    if hasattr(yolo_model, 'output_shape'):
        print(f"   • Output shape: {yolo_model.output_shape}")
    
    # Test with dummy input to see actual shapes
    dummy_input = tf.random.normal([1, 608, 608, 3])
    dummy_output = yolo_model(dummy_input)
    print(f"   • Actual input shape: {dummy_input.shape}")
    print(f"   • Actual output shape: {dummy_output.shape}")
    
except Exception as e:
    print(f"❌ Could not get shape information: {e}")

print()

# Model parameters count
try:
    print("🔢 Model Parameters:")
    if hasattr(yolo_model, 'count_params'):
        total_params = yolo_model.count_params()
        print(f"   • Total parameters: {total_params:,}")
        
        # Try to get trainable/non-trainable split
        try:
            trainable_params = sum([tf.keras.backend.count_params(w) for w in yolo_model.trainable_weights])
            non_trainable_params = sum([tf.keras.backend.count_params(w) for w in yolo_model.non_trainable_weights])
            print(f"   • Trainable parameters: {trainable_params:,}")
            print(f"   • Non-trainable parameters: {non_trainable_params:,}")
        except:
            print("   • Parameter breakdown not available")
    else:
        print("   • Parameter count not available for this model type")
        
except Exception as e:
    print(f"❌ Could not count parameters: {e}")

print()

# Layer information
try:
    print("🏗️ Layer Information:")
    if hasattr(yolo_model, 'layers'):
        print(f"   • Number of layers: {len(yolo_model.layers)}")
        print("   • Layer types:")
        for i, layer in enumerate(yolo_model.layers[:10]):  # Show first 10 layers
            print(f"     {i+1}. {layer.name}: {type(layer).__name__}")
        if len(yolo_model.layers) > 10:
            print(f"     ... and {len(yolo_model.layers) - 10} more layers")
    else:
        print("   • Layer information not available")
        
except Exception as e:
    print(f"❌ Could not get layer information: {e}")

print()

# For actual YOLO model, this would show the architecture
print("💡 Note about YOLO Architecture:")
print("   • A real YOLO model typically has:")
print("     - Convolutional backbone (like Darknet)")
print("     - Feature pyramid network")
print("     - Detection heads for different scales")
print("     - Outputs: box coordinates, confidence, class probabilities")
print("   • Input: RGB images (608×608×3)")
print("   • Output: Detections for multiple object classes")

=== YOLO Model Architecture Summary ===

🔍 Model Type Analysis:
   • Model class: <class 'keras.src.models.functional.Functional'>
   • Model name: functional_2

📋 Model Summary:
--------------------------------------------------


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 608, 608, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 3)              │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4 (16.00 B)

 Trainable params: 4 (16.00 B)

 Non-trainable params: 0 (0.00 B)

--------------------------------------------------

📐 Input/Output Information:
   • Input shape: (None, 608, 608, 3)
   • Output shape: (None, 1)
   • Actual input shape: (1, 608, 608, 3)
   • Actual output shape: (1, 1)

🔢 Model Parameters:
   • Total parameters: 4
   • Trainable parameters: 4
   • Non-trainable parameters: 0

🏗️ Layer Information:
   • Number of layers: 3
   • Layer types:
     1. input_layer_2: InputLayer
     2. global_average_pooling2d_2: GlobalAveragePooling2D
     3. dense_2: Dense

💡 Note about YOLO Architecture:
   • A real YOLO model typically has:
     - Convolutional backbone (like Darknet)
     - Feature pyramid network
     - Detection heads for different scales
     - Outputs: box coordinates, confidence, class probabilities
   • Input: RGB images (608×608×3)
   • Output: Detections for multiple object classes


In [22]:
# 🎯 Real YOLO Model Architecture Explanation

print("=== What a Real YOLO Model Would Look Like ===")
print()

print("🏗️ Typical YOLO Architecture:")
print("   1. INPUT: 608×608×3 RGB image")
print("   2. BACKBONE: Darknet-53 (or similar CNN)")
print("      • 53 convolutional layers")
print("      • Residual connections")
print("      • Feature extraction at multiple scales")
print("   3. NECK: Feature Pyramid Network (FPN)")
print("      • Combines features from different scales")
print("      • Upsampling and concatenation")
print("   4. HEAD: Detection layers")
print("      • 3 different scales (13×13, 26×26, 52×52)")
print("      • Each scale predicts: [x, y, w, h, confidence, class_probs]")
print("   5. OUTPUT: Object detections")
print("      • Bounding boxes")
print("      • Confidence scores")
print("      • Class probabilities")
print()

print("📊 Real YOLO Model Stats:")
print("   • Parameters: ~60+ million")
print("   • Layers: 100+ layers")
print("   • Model size: ~200+ MB")
print("   • Training: Pre-trained on COCO dataset")
print("   • Classes: 80 object categories")
print()

print("🔗 Current Placeholder vs Real Model:")
print("   Current placeholder:")
print(f"      • Parameters: {yolo_model.count_params():,}")
print(f"      • Layers: {len(yolo_model.layers)}")
print(f"      • Purpose: Structure testing only")
print()
print("   Real YOLO model:")
print("      • Parameters: ~60,000,000+")
print("      • Layers: 100+")
print("      • Purpose: Actual object detection")
print()

print("💡 To get the real model:")
print("   1. Fix Git LFS: git lfs install && git lfs pull")
print("   2. Or download from: https://pjreddie.com/media/files/yolov3.weights")
print("   3. Convert to Keras format if needed")
print()

# Optional: Create a more realistic placeholder architecture
print("🛠️ Would you like a more realistic placeholder architecture?")
print("   (This would be for educational purposes - still not trained)")

def create_realistic_yolo_placeholder():
    """Create a more realistic YOLO architecture (untrained)"""
    print("\n🏗️ Creating realistic YOLO placeholder...")
    
    # Input
    inputs = tf.keras.Input(shape=(608, 608, 3))
    
    # Simplified backbone (much smaller than real Darknet)
    x = tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu')(inputs)
    x = tf.keras.layers.Conv2D(64, 3, strides=2, padding='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D(128, 3, strides=2, padding='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D(256, 3, strides=2, padding='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D(512, 3, strides=2, padding='same', activation='relu')(x)
    
    # Detection head (simplified)
    # Real YOLO would have multiple scales, we'll do one for simplicity
    x = tf.keras.layers.Conv2D(1024, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D(512, 1, padding='same', activation='relu')(x)
    
    # Output: 19x19 grid, 5 boxes per cell, 85 values per box (4 coords + 1 conf + 80 classes)
    predictions = tf.keras.layers.Conv2D(5 * 85, 1, padding='same')(x)
    
    # Reshape to proper format
    predictions = tf.keras.layers.Reshape((19, 19, 5, 85))(predictions)
    
    model = tf.keras.Model(inputs=inputs, outputs=predictions)
    return model

# Uncomment the next line if you want to create the realistic placeholder
# realistic_yolo = create_realistic_yolo_placeholder()
# realistic_yolo.summary()

print("✅ Architecture explanation completed!")

=== What a Real YOLO Model Would Look Like ===

🏗️ Typical YOLO Architecture:
   1. INPUT: 608×608×3 RGB image
   2. BACKBONE: Darknet-53 (or similar CNN)
      • 53 convolutional layers
      • Residual connections
      • Feature extraction at multiple scales
   3. NECK: Feature Pyramid Network (FPN)
      • Combines features from different scales
      • Upsampling and concatenation
   4. HEAD: Detection layers
      • 3 different scales (13×13, 26×26, 52×52)
      • Each scale predicts: [x, y, w, h, confidence, class_probs]
   5. OUTPUT: Object detections
      • Bounding boxes
      • Confidence scores
      • Class probabilities

📊 Real YOLO Model Stats:
   • Parameters: ~60+ million
   • Layers: 100+ layers
   • Model size: ~200+ MB
   • Training: Pre-trained on COCO dataset
   • Classes: 80 object categories

🔗 Current Placeholder vs Real Model:
   Current placeholder:
      • Parameters: 4
      • Layers: 3
      • Purpose: Structure testing only

   Real YOLO model:
      • P

In [23]:
yolo_model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 608, 608, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 3)              │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4 (16.00 B)

 Trainable params: 4 (16.00 B)

 Non-trainable params: 0 (0.00 B)

In [24]:
def predict(image_file):
    """
    Runs the graph to predict boxes for "image_file". Prints and plots the predictions.
    
    Arguments:
    image_file -- name of an image stored in the "images" folder.
    
    Returns:
    out_scores -- tensor of shape (None, ), scores of the predicted boxes
    out_boxes -- tensor of shape (None, 4), coordinates of the predicted boxes
    out_classes -- tensor of shape (None, ), class index of the predicted boxes
    
    Note: "None" actually represents the number of predicted boxes, it varies between 0 and max_boxes. 
    """

    # Preprocess your image
    image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))
    
    yolo_model_outputs = yolo_model(image_data) # It's output is of shape (m, 19, 19, 5, 85) 
    # But yolo_eval takes input a tensor contains 4 tensors: box_xy,box_wh, box_confidence & box_class_probs
    yolo_outputs = yolo_head(yolo_model_outputs, anchors, len(class_names))
    
    out_scores, out_boxes, out_classes = yolo_eval(yolo_outputs, [image.size[1],  image.size[0]], 10, 0.3, 0.5)

    # Print predictions info
    print('Found {} boxes for {}'.format(len(out_boxes), "images/" + image_file))
    # Generate colors for drawing bounding boxes.
    colors = get_colors_for_classes(len(class_names))
    # Draw bounding boxes on the image file
    #draw_boxes2(image, out_scores, out_boxes, out_classes, class_names, colors, image_shape)
    draw_boxes(image, out_boxes, out_classes, class_names, out_scores)
    # Save the predicted bounding box on the image
    image.save(os.path.join("out", str(image_file).split('.')[0]+"_annotated." +str(image_file).split('.')[1] ), quality=100)
    # Display the results in the notebook
    output_image = Image.open(os.path.join("out", str(image_file).split('.')[0]+"_annotated." +str(image_file).split('.')[1] ))
    imshow(output_image)

    return out_scores, out_boxes, out_classes

In [25]:
out_scores, out_boxes, out_classes = predict("0001.jpg")

InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} slice index 1 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/